In [ ]:
%%writefile pix2pix_model.py
import os
import json

import numpy as np
import tensorflow as tf
from datetime import datetime

from numpy import zeros
from numpy import ones
from numpy.random import randint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import plot_model
from matplotlib import pyplot as plt

def define_discriminator(image_shape):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    
    # ảnh source
    in_src_image = Input(shape=image_shape)  
    # ảnh target
    in_target_image = Input(shape=image_shape) 

    # kết nối images, [256,256,6]
    merged = Concatenate()([in_src_image, in_target_image])

    # C64: 4x4 kernel Stride 2x2
    d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)
    # C128: 4x4 kernel Stride 2x2
    d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # C256: 4x4 kernel Stride 2x2
    d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # C512: 4x4 kernel Stride 2x2
    d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # 4x4 kernel but Stride 1x1
    d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # patch output
    d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
    patch_out = Activation('sigmoid')(d)
    # define model
    model = Model([in_src_image, in_target_image], patch_out)
    # compile model
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
    return model

def define_encoder_block(layer_in, n_filters, batchnorm=True):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # add downsampling layer
    g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
    # conditionally add batch normalization
    if batchnorm:
        g = BatchNormalization()(g, training=True)
    # leaky relu activation
    g = LeakyReLU(alpha=0.2)(g)
    return g

def decoder_block(layer_in, skip_in, n_filters, dropout=True):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # add upsampling layer
    g = Conv2DTranspose(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
    # add batch normalization
    g = BatchNormalization()(g, training=True)
    # conditionally add dropout
    if dropout:
        g = Dropout(0.5)(g, training=True)
    # merge with skip connection
    g = Concatenate()([g, skip_in])
    # relu activation
    g = Activation('relu')(g)
    return g

def define_generator(image_shape=(256,256,3)):
    # khởi tạo trọng số
    init = RandomNormal(stddev=0.02)
    # ảnh đầu vào
    in_image = Input(shape=image_shape)
    # encoder model: C64-C128-C256-C512-C512-C512-C512-C512
    e1 = define_encoder_block(in_image, 64, batchnorm=False)
    e2 = define_encoder_block(e1, 128)
    e3 = define_encoder_block(e2, 256)
    e4 = define_encoder_block(e3, 512)
    e5 = define_encoder_block(e4, 512)
    e6 = define_encoder_block(e5, 512)
    e7 = define_encoder_block(e6, 512)
    # bottleneck, no batch norm and relu
    b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e7)
    b = Activation('relu')(b)
    # decoder model: CD512-CD512-CD512-C512-C256-C128-C64
    d1 = decoder_block(b, e7, 512)
    d2 = decoder_block(d1, e6, 512)
    d3 = decoder_block(d2, e5, 512)
    d4 = decoder_block(d3, e4, 512, dropout=False)
    d5 = decoder_block(d4, e3, 256, dropout=False)
    d6 = decoder_block(d5, e2, 128, dropout=False)
    d7 = decoder_block(d6, e1, 64, dropout=False)
    # output
    g = Conv2DTranspose(image_shape[2], (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d7)
    out_image = Activation('tanh')(g)
    # define model
    model = Model(in_image, out_image)
    return model

def define_gan(g_model, d_model, image_shape):
    # make weights in the discriminator not trainable
    for layer in d_model.layers:
        if not isinstance(layer, BatchNormalization):
            layer.trainable = False

    # define the source image
    in_src = Input(shape=image_shape)
    # supply the image as input to the generator
    gen_out = g_model(in_src)
    # supply the input image and generated image as inputs to the discriminator
    dis_out = d_model([in_src, gen_out])
    # src image as input, generated image and disc. output as outputs
    model = Model(in_src, [dis_out, gen_out])
    # compile model
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1,100])
    return model

def generate_real_samples(src_batch, tar_batch, n_patch):
    batch_size = len(src_batch)
    # Create target "real" class labels (1s)
    y = ones((batch_size, n_patch, n_patch, 1))
    return [src_batch, tar_batch], y

def generate_fake_samples(g_model, src_batch, n_patch):
    # generate fake instance
    X = g_model.predict(src_batch, verbose=0)
    # create 'fake' class labels (0)
    y = zeros((len(X), n_patch, n_patch, 1))
    return X, y

def summarize_performance(step, g_model, data_generator, n_samples=3):
    """
    Tóm tắt hiệu suất của mô hình bằng cách tạo và hiển thị ảnh
    """
    # Lấy một batch ngẫu nhiên từ data generator
    batch_idx = np.random.randint(0, len(data_generator))
    src_batch, tar_batch = data_generator[batch_idx]
    
    # Chọn n_samples ảnh đầu tiên từ batch
    n_samples = min(n_samples, src_batch.shape[0])
    X_realA = src_batch[:n_samples]
    X_realB = tar_batch[:n_samples]
    
    # Tạo ảnh fake từ generator
    X_fakeB = g_model.predict(X_realA, verbose=0)
    
    # Chuẩn hóa pixel values từ [-1,1] về [0,1] để hiển thị
    X_realA = (X_realA + 1) / 2.0
    X_realB = (X_realB + 1) / 2.0
    X_fakeB = (X_fakeB + 1) / 2.0
    
    # Tạo figure với kích thước phù hợp
    plt.figure(figsize=(n_samples * 3, 9))
    
    # Hiển thị ảnh source (hàng 1)
    for i in range(n_samples):
        plt.subplot(3, n_samples, 1 + i)
        plt.axis('off')
        plt.title('Source', fontsize=10)
        if X_realA[i].shape[-1] == 1:  # Grayscale
            plt.imshow(X_realA[i].squeeze(), cmap='gray')
        else:  # RGB
            plt.imshow(X_realA[i])
    
    # Hiển thị ảnh generated (hàng 2)
    for i in range(n_samples):
        plt.subplot(3, n_samples, 1 + n_samples + i)
        plt.axis('off')
        plt.title('Generated', fontsize=10)
        if X_fakeB[i].shape[-1] == 1:  # Grayscale
            plt.imshow(X_fakeB[i].squeeze(), cmap='gray')
        else:  # RGB
            plt.imshow(X_fakeB[i])
    
    # Hiển thị ảnh target thực (hàng 3)
    for i in range(n_samples):
        plt.subplot(3, n_samples, 1 + n_samples*2 + i)
        plt.axis('off')
        plt.title('Target', fontsize=10)
        if X_realB[i].shape[-1] == 1:  # Grayscale
            plt.imshow(X_realB[i].squeeze(), cmap='gray')
        else:  # RGB
            plt.imshow(X_realB[i])
    
    # Điều chỉnh layout và lưu ảnh
    plt.tight_layout()
    filename1 = 'plot_epoch_%03d.png' % ((step // len(data_generator)) + 1)
    plt.savefig(filename1, dpi=150, bbox_inches='tight')
    plt.close()
    
    # Lưu mô hình generator
    filename2 = 'g_model_epoch_%03d.h5' % ((step // len(data_generator)) + 1)
    g_model.save(filename2)
    
    print(f'[INFO] Saved: {filename1} and {filename2}')

def train(d_model, g_model, gan_model, data_generator, n_epochs=100, checkpoint_dir='checkpoints'):
    # tạo thư mục checkpoint nếu chưa có
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
        
    # Create log directory for TensorBoard
    log_dir = f'logs/{datetime.now().strftime("%Y%m%d-%H%M%S")}'
    summary_writer = tf.summary.create_file_writer(log_dir)
    
    # file lưu trạng thái epoch
    checkpoint_file = os.path.join(checkpoint_dir, 'checkpoint.json')
    start_epoch = 0
    
    # nếu đã có checkpoint trước đó, load lại
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, 'r') as f:
            checkpoint = json.load(f)
            start_epoch = checkpoint.get('epoch', 0)
            print(f"[INFO] Resuming from epoch {start_epoch + 1}")
            # load lại trọng số mô hình
            if os.path.exists(os.path.join(checkpoint_dir, 'd_model.weights.h5')):
                d_model.load_weights(os.path.join(checkpoint_dir, 'd_model.weights.h5'))
                g_model.load_weights(os.path.join(checkpoint_dir, 'g_model.weights.h5'))
                gan_model.load_weights(os.path.join(checkpoint_dir, 'gan_model.weights.h5'))
            
    # lấy chiều cao trong discriminator (None, 16, 16, 1)
    n_patch = d_model.output_shape[1]
    # tính toán mỗi batch trong epoch
    bat_per_epo = len(data_generator)
    
    print(f"[INFO] Starting training from epoch {start_epoch + 1}")
    print(f"[INFO] Patch shape: {n_patch}")
    print(f"[INFO] Batches per epoch: {bat_per_epo}")

    # enumerate epochs
    for epoch in range(start_epoch, n_epochs):
        # Shuffle data at start of epoch
        data_generator.on_epoch_end()
        
        for batch in range(bat_per_epo):
            # current step
            step = epoch * bat_per_epo + batch + 1
            # Get batch data từ generator
            src_batch, tar_batch = data_generator[batch]

            # Generate real samples
            [X_realA, X_realB], y_real = generate_real_samples(src_batch, tar_batch, n_patch)
            
            # Generate fake samples
            X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)
            
            # Update discriminator for real samples
            d_loss1 = d_model.train_on_batch([X_realA, X_realB], y_real)
            
            # Update discriminator for fake samples
            d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
            
            # Update generator
            g_loss, _, _ = gan_model.train_on_batch(X_realA, [y_real, X_realB])
            
            print('STEP : %d, Epoch : %d, Batch : %d/%d,  Discriminator_real[%.3f] Discriminator_fake[%.3f] Generator[%.3f]' % (step, epoch+1, batch+1, bat_per_epo, d_loss1, d_loss2, g_loss))
            
            # Ghi loss vào TensorBoard sau mỗi step
            with summary_writer.as_default():
                tf.summary.scalar('Loss/Discriminator_real', d_loss1, step=step)
                tf.summary.scalar('Loss/Discriminator_fake', d_loss2, step=step)
                tf.summary.scalar('Loss/Generator', g_loss, step=step)
                
        # Lưu checkpoint sau mỗi epoch
        d_model.save_weights(os.path.join(checkpoint_dir, 'd_model.weights.h5'))
        g_model.save_weights(os.path.join(checkpoint_dir, 'g_model.weights.h5'))
        gan_model.save_weights(os.path.join(checkpoint_dir, 'gan_model.weights.h5'))
        with open(checkpoint_file, 'w') as f:
            json.dump({'epoch': epoch + 1}, f)
            
        # Tóm tắt hiệu suất mỗi 10 epoch
        if (epoch + 1) % 10 == 0:
            summarize_performance(step, g_model, data_generator, n_samples=3)
            
    print(f"[INFO] Training completed after {n_epochs} epochs")

In [ ]:
## import thư viện
from os import listdir
from numpy import asarray, load
from numpy import vstack
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from numpy import savez_compressed
from matplotlib import pyplot
import numpy as np
import os
from pix2pix_model import define_discriminator, define_generator, define_gan, train

In [ ]:
## khởi tạo ImageDataGenerator

from tensorflow.keras.utils import Sequence

class ImageDataGenerator(Sequence):
    def __init__(self, path, batch_size=16, size=(256,512), max_images=10000, shuffle=True):
        self.path = path
        self.batch_size = batch_size
        self.size = size
        self.max_images = max_images
        self.shuffle = shuffle
        
        # Lấy danh sách file
        all_files = os.listdir(path)
        self.image_files = [f for f in all_files[:max_images] 
                           if f.lower().endswith(('.jpg', '.jpeg', '.png', '.gif'))]
        
        # Tạo indices cho shuffle
        self.indices = np.arange(len(self.image_files))
        if self.shuffle:
            np.random.shuffle(self.indices)
            
        print(f"Data Generator initialized: {len(self.image_files)} images, {len(self)} batches")
        
    def __len__(self):
        return len(self.image_files) // self.batch_size
    
    def __getitem__(self, batch_idx):
        # Lấy indices cho batch này
        start_idx = batch_idx * self.batch_size
        end_idx = (batch_idx + 1) * self.batch_size
        batch_indices = self.indices[start_idx:end_idx]
        
        src_batch, tar_batch = [], []
        
        for idx in batch_indices:
            filename = self.image_files[idx]
            full_path = os.path.join(self.path, filename)
            
            try:
                # Load và resize ảnh
                pixels = load_img(full_path, target_size=self.size)
                pixels = img_to_array(pixels)
                
                # Chia ảnh thành source và target
                sat_img = pixels[:, :self.size[0], :]  # Left half
                map_img = pixels[:, self.size[0]:, :]  # Right half
                
                src_batch.append(sat_img)
                tar_batch.append(map_img)
                
            except Exception as e:
                print(f"Error loading {filename}: {e}")
                continue
        
        # Preprocess data (scale to [-1,1])
        src_batch = np.array(src_batch)
        tar_batch = np.array(tar_batch)
        
        src_batch = (src_batch - 127.5) / 127.5
        tar_batch = (tar_batch - 127.5) / 127.5
        
        return src_batch, tar_batch
    
    def on_epoch_end(self):
        """Shuffle data sau mỗi epoch"""
        if self.shuffle:
            np.random.shuffle(self.indices)

In [ ]:
# Khởi tạo data generator
data_generator = ImageDataGenerator(
    path='/kaggle/input/edge2shoes/train',
    batch_size=16,
    size=(256, 512),  # height=256, width=512 (sẽ được chia thành 2 ảnh 256x256)
    max_images=50000,
    shuffle=True
)
print(f"Total batches: {len(data_generator)}")
print(f"Total images: {len(data_generator.image_files)}")

In [ ]:
# Test load một batch
try:
    sample_src, sample_tar = data_generator[0]
    print(f"Source batch shape: {sample_src.shape}")
    print(f"Target batch shape: {sample_tar.shape}")
    print(f"Data range: [{sample_src.min():.3f}, {sample_src.max():.3f}]")
    
    image_shape = sample_src.shape[1:]  # (256, 256, 3)
    print(f"Image shape for model: {image_shape}")
    
except Exception as e:
    print(f"Error testing generator: {e}")

In [ ]:
# Copy từ /kaggle/input/ về /kaggle/working nếu cần ghi thêm
import shutil
import zipfile
import os

src = '/kaggle/input/tamthoi1/checkpoints'
dst = '/kaggle/working/checkpoints'

if not os.path.exists(dst):
    shutil.copytree(src, dst)
    print("✅ Đã copy thư mục checkpoints từ input vào working")
else:
    print("📁 Thư mục checkpoints đã tồn tại trong working")

# src = '/kaggle/input/total11/logs_50000_261/kaggle/working/logs'
# dst = '/kaggle/working/logs'

# if not os.path.exists(dst):
#     shutil.copytree(src, dst)
#     print("✅ Đã copy thư mục logs từ input vào working")
# else:
#     print("📁 Thư mục logs đã tồn tại trong working")

In [ ]:
# huấn luyện 
from datetime import datetime
# Lấy image shape từ một batch đầu tiên
sample_src, sample_tar = data_generator[0]
image_shape = sample_src.shape[1:]  # (256, 256, 3)

# Define models
d_model = define_discriminator(image_shape)
g_model = define_generator(image_shape)
gan_model = define_gan(g_model, d_model, image_shape)

# Training
start_time = datetime.now()
print(f"Training started at: {start_time}")

train(d_model, g_model, gan_model, data_generator, 
      n_epochs=341, checkpoint_dir='/kaggle/working/checkpoints')

end_time = datetime.now()
execution_time = end_time - start_time
print(f"Training completed at: {end_time}")
print(f"Total execution time: {execution_time}")

#**KIỂM TRA**

**Xóa**

In [ ]:
# logs_path = '/kaggle/working/logs'

# if os.path.exists(logs_path):
#     shutil.rmtree(logs_path)
#     print("Đã xóa thư mục logs.")
# else:
#     print("Thư mục logs không tồn tại.")

In [ ]:
# checkpoint_path = '/kaggle/working/checkpoints'

# # Kiểm tra xem thư mục có tồn tại không, nếu có thì xóa
# if os.path.exists(checkpoint_path):
#     shutil.rmtree(checkpoint_path)
#     print("✅ Đã xóa thư mục 'checkpoints'")
# else:
#     print("❌ Thư mục 'checkpoints' không tồn tại")

In [ ]:
# print(os.listdir('/kaggle/working'))

In [ ]:
# print(os.listdir('/kaggle/input/aaaaaa'))
# print(os.listdir('/kaggle/input/aaaaaa/checkpoints/kaggle/working/checkpoints'))
# print(os.listdir('/kaggle/input/aaaaaa/logs/kaggle/working/logs/20250521-114829'))

In [ ]:
# # Liệt kê file trong thư mục checkpoints
# print("📂 File trong checkpoints:")
# print(os.listdir('/kaggle/working/checkpoints'))

# # Liệt kê file trong thư mục logs
# print("\n📂 File trong logs:")
# print(os.listdir('/kaggle/working/logs'))

In [ ]:
# #Test trained model on a few images...

# from keras.models import load_model
# from numpy.random import randint

In [ ]:
# model = load_model('/kaggle/input/sladkjasl/keras/default/1/model_420001.h5')

In [ ]:
# # plot source, generated and target images
# def plot_images(src_img, gen_img, tar_img):
# 	images = vstack((src_img, gen_img, tar_img))
# 	# scale from [-1,1] to [0,1]
# 	images = (images + 1) / 2.0
# 	titles = ['Source', 'Generated', 'Expected']
# 	# plot images row by row
# 	for i in range(len(images)):
# 		# define subplot
# 		pyplot.subplot(1, 3, 1 + i)
# 		# turn off axis
# 		pyplot.axis('off')
# 		# plot raw pixel data
# 		pyplot.imshow(images[i])
# 		# show title
# 		pyplot.title(titles[i])
# 	pyplot.show()



# [X1, X2] = dataset
# # select random example
# ix = randint(0, len(X1), 1)
# src_image, tar_image = X1[ix], X2[ix]
# # generate image from source
# gen_image = model.predict(src_image)
# # plot all three images
# plot_images(src_image, gen_image, tar_image)

In [ ]:
# # === BƯỚC 1: Load và xử lý ảnh ===
# import numpy as np
# from tensorflow.keras.preprocessing.image import load_img, img_to_array
# from matplotlib import pyplot
# from numpy import vstack

# sizev2 = (256, 512)
# img = load_img('/kaggle/input/testshoes/val/121_AB.jpg', target_size=sizev2)  # Ảnh ghép 2 phần
# pixelsv2 = img_to_array(img)

# # Cắt ảnh: ảnh bên trái là input, ảnh bên phải là ground truth
# src = pixelsv2[:, :sizev2[0], :]   # (256, 256, 3)
# tar = pixelsv2[:, sizev2[0]:, :]   # (256, 256, 3)
# datav2 = [src, tar]

# def preprocess_data(data):
#     # load compressed arrays
#     # unpack arrays
#     X1, X2 = data[0], data[1]
#     # scale from [0,255] to [-1,1]
#     X1 = (X1 - 127.5) / 127.5
#     X2 = (X2 - 127.5) / 127.5
#     return [X1, X2]

# datasetv2 = preprocess_data(datav2)

# # plot source, generated and target images
# def plot_images(src, gen, tar):
#     # Đảm bảo dữ liệu có đúng shape và kiểu
#     images = [src, gen, tar]
#     titles = ['Source', 'Generated', 'Expected']
    
#     # Tạo figure với kích thước phù hợp
#     pyplot.figure(figsize=(15, 5))
    
#     # plot images row by row
#     for i in range(len(images)):
#         # define subplot (1 hàng, 3 cột, subplot thứ i+1)
#         pyplot.subplot(1, 3, i + 1)
#         # turn off axis
#         pyplot.axis('off')
        
#         # Scale từ [-1,1] về [0,1] cho từng ảnh
#         img_scaled = (images[i] + 1) / 2.0
#         # Đảm bảo giá trị trong khoảng [0,1]
#         img_scaled = np.clip(img_scaled, 0, 1)
        
#         # plot raw pixel data
#         pyplot.imshow(img_scaled)
#         # show title
#         pyplot.title(titles[i])
    
#     pyplot.tight_layout()
#     pyplot.show()

# [X1, X2] = datasetv2
# src, tar = X1, X2

# # === PHẦN SỬA LỖI: Thêm batch dimension ===
# # Chuyển từ shape (256, 256, 3) thành (1, 256, 256, 3)
# src_batch = np.expand_dims(src, axis=0)

# # generate image from source
# gen_batch = model.predict(src_batch)

# # Loại bỏ batch dimension để về lại shape (256, 256, 3)
# gen = np.squeeze(gen_batch, axis=0)

# # plot all three images
# plot_images(src, gen, tar)

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np
# from tensorflow.keras.preprocessing.image import load_img, img_to_array

# # === Load và xử lý ảnh source ===
# img = load_img('/kaggle/input/aaaaasde/test2.png', target_size=(256, 256))
# img_array = img_to_array(img)

# print(f"Original image range: [{img_array.min():.2f}, {img_array.max():.2f}]")
# print(f"Original image shape: {img_array.shape}")

# # Scale từ [0,255] về [-1,1] giống lúc train
# img_array = (img_array - 127.5) / 127.5
# print(f"Normalized image range: [{img_array.min():.2f}, {img_array.max():.2f}]")

# # Thêm batch dimension
# src_image = np.expand_dims(img_array, axis=0)  # (1, 256, 256, 3)

# # === Dự đoán ===
# gen_image = model.predict(src_image, verbose=0)
# print(f"Generated image range: [{gen_image.min():.2f}, {gen_image.max():.2f}]")

# # === Vẽ ảnh Source và Generated ===
# def plot_images_debug(src_img, gen_img):
#     # scale từ [-1,1] về [0,1] để hiển thị
#     src_display = (src_img + 1) / 2.0
#     gen_display = (gen_img + 1) / 2.0
    
#     # Clip để đảm bảo trong khoảng [0,1]
#     src_display = np.clip(src_display, 0, 1)
#     gen_display = np.clip(gen_display, 0, 1)
    
#     images = [src_display[0], gen_display[0]]
#     titles = ['Source', 'Generated']
    
#     plt.figure(figsize=(12, 6))
#     for i in range(2):
#         plt.subplot(1, 2, i + 1)
#         plt.axis('off')
#         plt.title(titles[i])
#         plt.imshow(images[i])
        
#         # Thêm thông tin về range
#         img_min, img_max = images[i].min(), images[i].max()
#         plt.xlabel(f'Range: [{img_min:.3f}, {img_max:.3f}]')
    
#     plt.tight_layout()
#     plt.show()

# plot_images_debug(src_image, gen_image)

# # === Thêm histogram để so sánh phân phối ===
# def plot_histograms(src_img, gen_img):
#     plt.figure(figsize=(12, 4))
    
#     # Histogram của source image
#     plt.subplot(1, 2, 1)
#     plt.hist(src_img.flatten(), bins=50, alpha=0.7, color='blue')
#     plt.title('Source Image Histogram')
#     plt.xlabel('Pixel Value')
#     plt.ylabel('Frequency')
    
#     # Histogram của generated image
#     plt.subplot(1, 2, 2)
#     plt.hist(gen_img.flatten(), bins=50, alpha=0.7, color='red')
#     plt.title('Generated Image Histogram')
#     plt.xlabel('Pixel Value')
#     plt.ylabel('Frequency')
    
#     plt.tight_layout()
#     plt.show()

# plot_histograms(src_image, gen_image)

In [ ]:
!zip -r /kaggle/working/checkpoints.zip /kaggle/working/checkpoints
!zip -r /kaggle/working/logs.zip /kaggle/working/logs

In [ ]:
# import shutil

# # Copy file checkpoint
# shutil.copy('/kaggle/working/checkpoints/d_model.weights.h5', '/kaggle/working/d_model.weights.h5')
# shutil.copy('/kaggle/working/checkpoints/g_model.weights.h5', '/kaggle/working/g_model.weights.h5')
# shutil.copy('/kaggle/working/checkpoints/gan_model.weights.h5', '/kaggle/working/gan_model.weights.h5')
# shutil.copy('/kaggle/working/checkpoints/checkpoint.json', '/kaggle/working/checkpoint.json')

# # Copy file logs
# shutil.copytree('/kaggle/working/logs/20250521-114829', '/kaggle/working/logs_copy')


In [ ]:
# tensorboard --logdir="D:\AI\kaggle\working\logs\20250521-114829"